# Extract Whistles From Box 2 Box

We need to extract whistles played through the ocean and recorded by a box.

## Basic Idea
Use the original file to find gaps.
Align the recorded file to the original file (manually).
Find all gaps that are large enough in the original file:
    $$(gap_{i, start}, gap_{i, stop})$$.
    
Use the gaps to find whistles.

In [186]:
import os
import matplotlib.pyplot as plt
import numpy as np

from mimic_utils.spectrogram import *
from mimic_utils.whistle_tracer import *
from mimic_utils.params import * 
from scipy.io import wavfile
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw


## The algorithms to extract the audio

In [193]:
def wtrace(audio):
    spec = fwd_spectrogram(data_original[start - border: stop + border, 0], FFT_WIN, FFT_STEP)
    whistle_trace, _  = trace(spec, TRACE_RAD, SMOOTH_ENT)   
    lo = min(whistle_trace)
    hi = max(whistle_trace)
    return (whistle_trace - lo) / (hi - lo)


class Classifier:
    
    def __init__(self, folder):
        self.templates = {}
        for filename in os.listdir(folder):
            if filename.endswith(".wav"):
                print("Loading Classifier Template: {}".format(filename))
                basename = recorded.split("/")[-1].replace(".wav", "")
                _,  data = wavfile.read("{}/{}".format(folder, filename))
                self.templates[basename] = wtrace(data)
        print(self.templates)
        
    def nn(self, trace):
        min_label = -1
        min_dist  = float('inf') 
        for label, template in self.templates.items():
            dist, _ = fastdtw(trace, template, dist=euclidean)
            if dist < min_dist:
                min_dist = dist
                min_label = label
        return label            

    
def extract_all(original, recorded, output, classifier, border = 30000, min_gap_size = 123500, th = 0.1):
    basename = recorded.split("/")[-1].replace(".wav", "")
    print(basename)
    _,  data_original = wavfile.read(original)
    fs, data_box      = wavfile.read(recorded)
    data = (data_original[:, 0] + 32768) / (32768 + 32767)    

    last_sample = 0
    start_i = 0    
    gaps = []
    
    n = len(data_original)
    for i in range(10, 190000 * 5):
        sample = np.sum(data[i - 10:i]) / 10
        if sample < th and last_sample >= th and i - start_i > min_gap_size:
            print("STOP {} {} {} %done {} {}".format(start_i / fs, i / fs, (i - start_i) / fs, (i / n) * 100, len(gaps)))
            gaps.append([start_i, i])        
        if sample >= th and last_sample < th:
            start_i = i
        last_sample = sample
        
    for i in range(1, len(gaps)):  
        _, start = gaps[i - 1]
        stop, _  = gaps[i]
        trace = wtrace(data_original[start - border: stop + border, 0])
        name = classifier.nn(trace)
        filename = '{}/{}_{}_{}.wav'.format(output, basename, start, name)
        print(filename)
        wavfile.write(filename, fs, data_box[start - border: stop + border, 0])

## Extract the 00 variations

In [ ]:
classifier = Classifier("originals/")
output     = "00" 
original   = '/Users/daniel.kohlsdorf/Desktop/00-all-whistles-2019-synth--04--18.wav'
recorded   = '/Users/daniel.kohlsdorf/Desktop/chat1-2019-06-23T123304-192k.wav'
extract_all(original, recorded, output, classifier)

sar.wav


/Users/daniel.kohlsdorf/anaconda3/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


sca.wav
gra.wav
den.wav
